In [52]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import max_error, mean_squared_error, median_absolute_error, mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [22]:
import sys
sys.path.append("../../") 

In [53]:
def calcular_metricas(y_true, y_predict):
    return {
        "MAX_ERROR": max_error(y_true, y_predict),
        "MEAN_SQ_ERROR": mean_squared_error(y_true, y_predict),
        "MEDIAN_ABS_ERROR": median_absolute_error(y_true, y_predict),
        "MEAN_ABS_ERROR": mean_absolute_error(y_true, y_predict),
    }

In [1]:
def heuristica(Vviento):
    """Definimos la heurística basada en la Escala de Beaufort"""
    if Vviento < 3:
        return 0.4
    elif Vviento < 7:
        return 0.6
    elif Vviento < 12:
        return 1
    elif Vviento < 20:
        return 1.5
    elif Vviento < 26:
        return 2
    elif Vviento < 36:
        return 2.5
    elif Vviento < 46:
        return 3.2
    elif Vviento < 56:
        return 4.2
    elif Vviento < 66:
        return 5.4
    elif Vviento < 76:
        return 6.5
    elif Vviento < 88:
        return 8
    elif Vviento < 101:
        return 10
    elif Vviento >= 101:
        return 12

Cargamos los datos que hemos capturado hasta ahora

In [74]:
data = pd.read_csv(r"D:\Users\Mario\Downloads\PD\forecasts\clean\forecast_merged.csv")
data = data.dropna() #Quitamos todas aquellas filas que tengan algun na

In [76]:
X = data.drop(columns = "AlturaOlas")
X = X.drop(columns=["antelacion", 'anio', 'mes', 'dia', 'hora'])
y = data["AlturaOlas"]

In [77]:
"""X_model = X.copy()
scaler = ColumnTransformer(
    transformers=[
        # no escalonamos las dummy de dirección de viento
        ("num", StandardScaler(), [c for c in X_model.columns if not c.startswith('wdir_')]),
    ],
    remainder='passthrough'
)
scaler.fit(X_model)"""
X_model = scaler.transform(X_model);

In [78]:
X_model

array([[ 1.14775063, -0.94156517,  0.79894913, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.47783256, -0.94156517,  0.79894913, ...,  1.        ,
         0.        ,  0.        ],
       [-0.19208551, -0.46456577,  0.79894913, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 1.14775063,  0.96643244, -1.23524304, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.14775063,  0.96643244, -3.52911932, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.8176687 ,  0.96643244, -0.97555894, ...,  0.        ,
         0.        ,  0.        ]])

Cargamos el modelo

In [62]:
with open("D:/Users/Mario/Downloads/PD/modelos/modelo_definitivo/modelo_rf_def.pkl", 'rb') as file:
    model = pickle.load(file)
model

RandomForestRegressor(max_depth=28, min_samples_leaf=4, min_samples_split=4,
                      n_estimators=201, random_state=777)

In [80]:
predicciones_modelo = model.predict(X_model)
predicciones_heuristica = X["Vviento"].apply(heuristica)

C:\Users\Mario\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [81]:
predicciones_modelo

array([1.6667244 , 1.64522484, 1.64032195, 1.63854173, 1.63854173,
       1.64522484, 1.64522484, 1.53105601, 1.76156309, 1.64032195,
       1.64032195, 1.53105601, 1.53105601, 1.53105601, 1.64032195,
       1.53535143, 1.53105601, 1.53105601, 1.67879312, 1.67879312,
       1.6667244 , 1.6667244 , 1.6667244 , 1.67879312, 1.6667244 ,
       1.64522484, 1.65549187, 1.65549187, 1.53105601, 1.67879312,
       1.6667244 , 1.6667244 , 1.6667244 , 1.67879312, 1.6667244 ,
       1.64522484, 1.65549187, 1.65549187, 1.53105601, 1.53105601,
       1.53105601, 1.53105601, 1.53105601, 1.53105601, 1.53105601,
       1.53105601, 1.53105601, 1.53105601, 1.64522484, 1.53105601,
       1.64522484, 1.53105601, 1.64522484, 1.6667244 , 1.6667244 ,
       1.6667244 , 1.6667244 , 1.6667244 , 1.64522484, 1.6667244 ,
       1.64522484, 1.53105601, 1.64522484, 1.6801364 , 1.6801364 ,
       1.6801364 , 1.64522484, 1.53105601, 1.54341081, 1.64522484,
       1.53105601, 1.53105601, 1.53105601, 1.53105601, 1.53105

In [82]:
predicciones_heuristica

0      1.0
1      0.6
2      0.6
3      0.6
4      0.6
      ... 
478    1.0
482    1.0
485    1.0
489    1.0
495    1.0
Name: Vviento, Length: 422, dtype: float64

In [85]:
y

0      0.8
1      0.8
2      0.8
3      0.8
4      0.8
      ... 
478    0.9
482    0.7
485    0.9
489    0.7
495    0.7
Name: AlturaOlas, Length: 422, dtype: float64

In [88]:
calcular_metricas(y, predicciones_modelo)

{'MAX_ERROR': 1.1874677124731987,
 'MEAN_SQ_ERROR': 0.8648103925271358,
 'MEDIAN_ABS_ERROR': 0.9452248385698772,
 'MEAN_ABS_ERROR': 0.9029528437354865}

In [89]:
calcular_metricas(y, predicciones_heuristica)

{'MAX_ERROR': 0.6,
 'MEAN_SQ_ERROR': 0.0699526066350711,
 'MEDIAN_ABS_ERROR': 0.09999999999999998,
 'MEAN_ABS_ERROR': 0.19763033175355446}